# Science modules in Fink: an example

A science module contains necessary routines and classes to process the data, and add values. Typically, you will receive alerts in input, and output the same alerts with additional information. Input alert information contains position, flux, telescope properties, ... You can find what information is available in an alert [here](https://zwickytransientfacility.github.io/ztf-avro-alert/), or check the current Fink added values.

In this simple example, we explore a simple science module that takes magnitudes contained in each alert, and computes the change in magnitude between the last two measurements.

In [1]:
# utility from fink-science
from fink_science.utilities import concat_col

# user-defined function from the current folder
from processor import deltamaglatest

## Loading the data

Fink receives data as Avro. However, the internal processing makes use of Parquet files. We provide here alert data as Parquet: it contains original alert data from ZTF and some added values from Fink. You can download data at https://owncloud.lal.in2p3.fr/index.php/s/NAkl3Leqcgo02JC (85MB)

In [3]:
# Load the data into a Spark DataFrame
df = spark.read.format('parquet').load('20191101')

You can check what's in the data

In [4]:
df.printSchema()

root
 |-- schemavsn: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- objectId: string (nullable = true)
 |-- candid: long (nullable = true)
 |-- candidate: struct (nullable = true)
 |    |-- jd: double (nullable = true)
 |    |-- fid: integer (nullable = true)
 |    |-- pid: long (nullable = true)
 |    |-- diffmaglim: float (nullable = true)
 |    |-- pdiffimfilename: string (nullable = true)
 |    |-- programpi: string (nullable = true)
 |    |-- programid: integer (nullable = true)
 |    |-- candid: long (nullable = true)
 |    |-- isdiffpos: string (nullable = true)
 |    |-- tblid: long (nullable = true)
 |    |-- nid: integer (nullable = true)
 |    |-- rcid: integer (nullable = true)
 |    |-- field: integer (nullable = true)
 |    |-- xpos: float (nullable = true)
 |    |-- ypos: float (nullable = true)
 |    |-- ra: double (nullable = true)
 |    |-- dec: double (nullable = true)
 |    |-- magpsf: float (nullable = true)
 |    |-- sigmapsf: float (nullab

## Calling the science module

First, you need to concatenate historical + current measurements for the quantities of interest. Here, we only need `magpsf`. Hence we create a new column to the DataFrame called `cmagpsf` (for _concatenated_ `magpsf`):

In [5]:
# Required alert columns
what = ['magpsf']

# Use for creating temp name
prefix = 'c'
what_prefix = [prefix + i for i in what]

# Concatenate historical + current measurements
for colname in what:
    df = concat_col(df, colname, prefix=prefix)

Let's apply the science module, that is creating a new column to the DataFrame whose values are the change in magnitude between the last 2 measurements. All the user logic is contained in the routine `deltamaglatest` defined in `processor.py`. This routine is a user-defined function that encapsulates the necessary operations, and it can call functions from user-defined modules (here `mymodule.py`) or third-party libraries (e.g. `numpy`, `pandas`, etc). Note that the input arguments of `deltamaglatest` are column names of the DataFrame, and they are materialised as `pd.Series` inside the routine.

In [6]:
df_change = df.withColumn('deltamag', deltamaglatest('cmagpsf'))

# print the result for the 20 first alerts
df_change.select(['objectId', 'cdsxmatch', 'deltamag']).show()

+------------+---------+--------------------+
|    objectId|cdsxmatch|            deltamag|
+------------+---------+--------------------+
|ZTF19acmcetc|  Unknown|                null|
|ZTF17aaanypg|      YSO| -2.0131711959838867|
|ZTF19acmbxka|  Unknown|                null|
|ZTF19acmbxfe|  Unknown|                null|
|ZTF19acmbtac|  Unknown|                null|
|ZTF18acguzti|  Unknown|                null|
|ZTF19acmbxrc|  Unknown|                null|
|ZTF19acmbwej|  Unknown|                null|
|ZTF18acpvmvh|  Unknown|  0.6174602508544922|
|ZTF19acmbvze|  Unknown|                null|
|ZTF19acdtnex|  Unknown| -0.4819488525390625|
|ZTF18acgtyfa|  Unknown|                null|
|ZTF18aaacmcm|    RRLyr|   0.564335823059082|
|ZTF19acevxhy| Orion_V*|  0.1923046112060547|
|ZTF19acmbwaw|  Unknown|                null|
|ZTF19acmbxox|  Unknown|                null|
|ZTF19acgjjpl|  Unknown|-0.15512657165527344|
|ZTF18aabkgda|      EB*| -1.0305461883544922|
|ZTF17aabtkhd|  Unknown|     -1.42

We can also quickly check some statistics on this new column:

In [7]:
df_change.select('deltamag').describe().show()

+-------+--------------------+
|summary|            deltamag|
+-------+--------------------+
|  count|                 828|
|   mean|-0.46653374612043447|
| stddev|  0.7065887090053047|
|    min|  -4.440143585205078|
|    max|   2.567479133605957|
+-------+--------------------+



Et voilà! Of course, this science module is extremely simple - but the logic remains the same for more complex cases!